# PCN-Rec: Proof-Carrying Negotiation for Recommendation

This notebook runs the full pipeline for the "Proof-Carrying Negotiation" project.

**Steps:**
1. Setup (Install dependencies, Clone repo)
2. Step 1: Data Prep & Candidate Generation (LightFM)
3. Step 2: PCN-Rec Experiment (Single LLM vs PCN vs Ablations)
4. Evaluation & Results

**Requirements:**
- Google Gemini API Key

## 1. Setup

In [ ]:
# @title Install Dependencies
!pip install google-genai lightfm numpy pandas scipy pyyaml pyarrow tqdm scikit-learn tenacity pydantic

In [ ]:
# @title Clone Repository
import os
if not os.path.exists('pcn-rec'):
    !git clone https://github.com/Shreem1105/Proof-Carrying-Negotiation-for-Recommendation.git pcn-rec
os.chdir('pcn-rec')
print("Current working directory:", os.getcwd())

In [ ]:
# @title Set Gemini API Key
import os
from google.colab import userdata

try:
    os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
    print("API Key loaded from Secrets.")
except:
    print("API Key not found in Secrets. Please input below:")
    key = input("Enter Gemini API Key: ")
    os.environ["GEMINI_API_KEY"] = key

## 2. Step 1: Foundation (Data & Candidates)

In [ ]:
# Prepare Data (MovieLens 100K)
!python scripts/step1_prepare_data.py --config config/config.yaml --run_id paper_exp

In [ ]:
# Train Candidate Model (LightFM)
!python scripts/step1_train_candidates.py --config config/config.yaml --run_id paper_exp

In [ ]:
# Generate Top-100 Candidates (for all users)
!python scripts/step1_generate_candidates.py --config config/config.yaml --run_id paper_exp

## 3. Step 2: PCN-Rec Experiment

We will run the experiment for a subset of users to demonstrate functionality. For the full paper results, remove `--max_users`.

In [ ]:
# CONFIGURATION
MAX_USERS = 50 # Set to None for full run (WARNING: Takes time and API quota)
RUN_ID = "paper_exp"

In [ ]:
# 1. Baseline: Single LLM Reranker
!python scripts/step2_run_single_llm_baseline.py --config config/config.yaml --run_id {RUN_ID} --max_users {MAX_USERS}

In [ ]:
# 2. PCN-Rec (Negotiation + Verifier)
!python scripts/step2_run_pcnrec.py --config config/config.yaml --run_id {RUN_ID} --max_users {MAX_USERS}

In [ ]:
# 3. Ablations
!python scripts/step2_run_ablations.py --config config/config.yaml --run_id {RUN_ID} --max_users {MAX_USERS}

## 4. Evaluation & Results

In [ ]:
!python scripts/step2_evaluate.py --config config/config.yaml --run_id {RUN_ID}

In [ ]:
import pandas as pd
import os

summary_path = f"outputs/{RUN_ID}/evaluation_summary.csv"
if os.path.exists(summary_path):
    df = pd.read_csv(summary_path)
    display(df)
else:
    print("Evaluation summary not found.")